# Plot the KP turbine wakes (hub-height)

In [1]:
# Load the amrwind-frontend module
# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['../',
                 '/ccs/proj/cfd162/lcheung/amrwind-frontend/']
import sys, os, shutil
for x in amrwindfedirs: sys.path.insert(1, x)

# Important header information                                                                                                                                               
import postprolib as pp
# Load the libraries                                                                                                                                                         
import postproamrwindsample as ppsample
import numpy             as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr
import pickle
import pandas as pd

import classifyKPturbines
import ruamel.yaml as yaml
#from collections            import OrderedDict
#from ruamel.yaml.comments import CommentedMap as ordereddict
from ruamel.yaml.comments import CommentedMap as OrderedDict

# Make all plots inline 
%matplotlib inline

In [2]:
loaderkwargs = {'Loader':yaml.RoundTripLoader}
dumperkwargs = {'Dumper':yaml.RoundTripDumper, 'indent':2, 'default_flow_style':False}
from ruamel.yaml.comments import CommentedMap 
def comseq(d):
    """
    Convert OrderedDict to CommentedMap
    """
    if isinstance(d, OrderedDict):
        cs = CommentedMap()
        for k, v in d.items():
            cs[k] = comseq(v)
        return cs
    return d

In [3]:
extractvar = lambda xrds, var, i : xrds[var][i,:].data.reshape(tuple(xrds.attrs['ijk_dims'][::-1]))

def loadPickle(picklefname):
    pfile = open(picklefname, 'rb')
    db   = pickle.load(pfile)
    x    = db['x']
    y    = db['y']
    z    = db['z']
    vx   = db['vx']
    vy   = db['vy']
    vz   = db['vz']
    time = db['time']
    pfile.close()
    return x, y, z, vx, vy, vz, time

def avgfield(v, mintime, maxtime):
    avgv = None
    iavg = 0
    for itime, vfield in v.items():
        if (mintime<=itime) and (itime<=maxtime):
            iavg += 1
            if avgv is None:
                avgv = vfield
            else:
                avgv += vfield
    #print(iavg)
    return avgv/float(iavg)

def makecbarax(ax, c, fontsize, size='5%'):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size=size, pad=0.05)
    cbar=fig.colorbar(c, ax=ax, cax=cax)
    cbar.ax.tick_params(labelsize=fontsize)

def setfigtextsize(ax, fsize):
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, ax.yaxis.get_offset_text()] + ax.get_xticklabels() + ax.get_yticklabels() ):
        item.set_fontsize(fsize)

In [4]:
def getCartesianMesh(x,y):
    p00=np.array([x[0,0], y[0,0]])
    p10=np.array([x[1,0], y[1,0]])
    p01=np.array([x[0,1], y[0,1]])
    dx=np.linalg.norm(p00-p10)
    dy=np.linalg.norm(p00-p01)
    hhshape=x.shape
    xvec=np.arange(hhshape[0])*dx
    yvec=np.arange(hhshape[1])*dy
    xm, ym = np.meshgrid(xvec, yvec)
    return xm.transpose(), ym.transpose()

def avgFileList(turblist, tavg1, tavg2):
    avgvx = None
    avgvy = None
    avgvz = None
    for turb in turblist:
        x, y, z, vx, vy, vz, time = loadPickle(turb)
        vx_avg = avgfield(vx, tavg1, tavg2)
        vy_avg = avgfield(vy, tavg1, tavg2)
        vz_avg = avgfield(vz, tavg1, tavg2)
        if avgvx is None:
            avgvx = vx_avg
            avgvy = vy_avg
            avgvz = vz_avg
        else:
            avgvx += vx_avg
            avgvy += vy_avg
            avgvz += vz_avg
    N = len(turblist)
    #print(time)
    return x, y, z, avgvx/N, avgvy/N, avgvz/N

def extractAvgCL(pklfile, tavg1, tavg2, ix, rotorscale=1.0):
    x, y, z, vx, vy, vz, time = loadPickle(pklfile)
    vx_avg = avgfield(vx, tavg1, tavg2)
    vy_avg = avgfield(vy, tavg1, tavg2)
    vz_avg = avgfield(vz, tavg1, tavg2) 
    xm, ym = getCartesianMesh(x,y)
    x0 = np.mean(xm)
    y0 = np.mean(ym)
    xp, yp = (xm-x0)/rotorscale, (ym-y0)/rotorscale
    xD = xp[:,0]
    yD = yp[0,:]
    Qvx = vx_avg[0,:,:]
    Qvy = vy_avg[0,:,:]
    Qvz = vz_avg[0,:,:]
    return yD, Qvx[ix,:], Qvy[ix,:], Qvz[ix,:]

In [5]:
def writeCLfiles(groupdict, tavg1, tavg2, ix, filename_template, headerdict={}):
    outputyaml=OrderedDict()
    for k,g in headerdict.items():
        outputyaml[k] = g
    
    allwturbfiles=[]
    allnoturbfiles=[]
    # Run through the turbines (with turbines)
    for iturb, turb in enumerate(groupdict['turblist']):
        header="X VX VY VY VZ"
        
        y, vx, vy, vz = extractAvgCL(groupdict['wturbfiles'][iturb], tavg1, tavg2, ix)
        dat = np.vstack((y, vx, vy, vz)).transpose()
        replacedict={'turb':turb, 'type':'wturb'}
        filename=filename_template.format(**replacedict)
        #'KP_wake_profiles/{turb}/{turb}_CL_{type}.dat'.format(**replacedict)
        dirpath = os.path.dirname(filename)
        try:
            os.makedirs(dirpath)
        except:
            pass
        np.savetxt(filename, dat, header=header)
        wturbfiles=OrderedDict()
        wturbfiles['name']=turb
        wturbfiles['centerline']=filename
        
        y, vx, vy, vz = extractAvgCL(groupdict['noturbfiles'][iturb], tavg1, tavg2, ix)
        dat = np.vstack((y, vx, vy, vz)).transpose()
        replacedict={'turb':turb, 'type':'noturb'}
        filename=filename_template.format(**replacedict)
        dirpath = os.path.dirname(filename)
        try:
            os.makedirs(dirpath)
        except:
            pass
        np.savetxt(filename, dat, header=header)
        noturbfiles=OrderedDict()
        noturbfiles['name']=turb
        noturbfiles['centerline']=filename
        
        # Add to yaml
        allwturbfiles.append(wturbfiles)
        allnoturbfiles.append(noturbfiles)
    outputyaml['turbinerun']=allwturbfiles
    outputyaml['precursorrun']=allnoturbfiles
    return outputyaml

In [6]:
summitcsv    = os.path.abspath('../UnstableABL_farmrun_turbines.csv')
pkldir       = os.path.abspath('means/KPturbshh/')
turbname     = '%s_hh_mean_%s.pkl'
tavg1        = 300
tavg2        = 961
rotorD       = 1
datadir      = '../../CompareResults/TurbineWakes/Unstable/DATA_Summit_amrwind_bananasrun1'
yamlfile     = 'KingPlainsCenterlines.yaml'

In [7]:
allturblist = classifyKPturbines.getturbnames(classifyKPturbines.getTurbSubset(summitcsv, '-KP'))

In [8]:
allturbgroups = [{'label':'All turbines',
                  'turblist':classifyKPturbines.getturbnames(classifyKPturbines.getTurbSubset(summitcsv, '-KP')),
                 },
                 ]


In [9]:
# Add the file lists
for turbgroup in allturbgroups:
    turbgroup['wturbfiles']  = [pkldir+'/'+turbname%(x, 'wturb') for x in turbgroup['turblist']]
    turbgroup['noturbfiles'] = [pkldir+'/'+turbname%(x, 'noturb') for x in turbgroup['turblist']]
    

In [10]:
# Go to the data directory
cwd              = os.getcwd()
if not os.path.exists(datadir):
    os.makedirs(datadir)
os.chdir(datadir)

header=OrderedDict({'title':'King Plains turbine wake profiles', 'avgover':[tavg1, tavg2]})
outyaml = writeCLfiles(allturbgroups[0], tavg1, tavg2, 13, '{turb}/{turb}_CL_{type}.dat', headerdict=header)

with open(yamlfile, 'w') as f:
    yaml.dump(comseq(outyaml), f, **dumperkwargs)
    
# Go back to the original directory
os.chdir(cwd)